# Handling Missing Data

Assumption: the data you will be given is clean and tidy



Reality: It is usually an utter mess

Get two or more sources of data on a similar domain and the problems unfolds further

# Trade-Offs in Missing Data Conventions

Strategies for handling missing data in general revolve around two approaches:

1. *Masking* the missing values
2. Choosing a *sentinel value* indicative of a missing entry

### Examples of masks

- Could be a Boolean array
- Could be an indicative value in the data of the null status e.g a bit

### Examples of sentinel values

- A data specific convention e.g -9999
- Not a Number (NaN)

### Trade-offs

- Masks: requires allocation of a Boolean array which adds overhead in terms of storage and computation
- Sentinel values: reduces the range of values that can be represented.

## Missing Data in Pandas

Constrained by reliance on NumPy.

NumPy has no built-in notion of not a number for non-floating point data types.

NumPy does support masked arrays (`numpy.ma`) - but this also comes with overheads.

So, Pandas uses sentinels for missing data in particular Python's `NaN` value and the Python `None` object.

## `None`: Pythonic missing data

A Python singleton object

Can be used in arrays with data type `object`

In [1]:
import numpy as np
import pandas as pd

vals1 = np.array([1, None, 3, 4])
vals1

array([1, None, 3, 4], dtype=object)

`dtype=object` is the best common type of representation here.

Generally, get an error when attempting to perform aggregations.

In [2]:
vals1.sum()

TypeError: ignored

## `NaN`: Missing numerical data

A special floating-point value recognized by all systems that use the standard IEEE floating-point representation:

In [3]:
vals2 = np.array([1, np.nan, 3, 4])
vals2.dtype

dtype('float64')

In [4]:
1 + np.nan

nan

In [5]:
0 *  np.nan

nan

In [6]:
vals2.sum(), vals2.min(), vals2.max()

(nan, nan, nan)

(hey jude)

In [7]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

## NaN and None in Pandas

`NaN` and `None` both have their place, and Pandas is built to handle the two of them nearly interchangeably, converting between them where appropriate:

In [8]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

For types without a sentinel value, Pandas automatically typecasts in the presence of NaN values.

E.g in an integer array if we set a value to `np.nan` it will automatically be upcast to floating-point types to accomodate it:

In [9]:
x = pd.Series(range(2), dtype=int)
x

0    0
1    1
dtype: int64

In [10]:
x[0] = None
x

0    NaN
1    1.0
dtype: float64

## Operating on Null Values

Given the presence of null values how do we detect, remove and replace these values in Pandas data structures?

## Detecting null values

Pandas data structures have two useful methods for detecting null data: `isnull()` and `notnull()`. Either one will return a Boolean mask over the data. For example:

In [11]:
data = pd.Series([1, np.nan, 'hello', None])

In [12]:
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

Boolean masks can be used directly as a `Series` or `DataFrame` index:

In [13]:
data[data.notnull()]

0        1
2    hello
dtype: object

## Dropping null values

In addition to the masking used before, there are the convenience methods, `dropna()` (which removes NA values) and `fillna()` (which fills in NA values):

In [14]:
data.dropna()

0        1
2    hello
dtype: object

For a `DataFrame`, there are more options:

In [15]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [16]:
df.dropna()

,0,1,2
1,2.0,3.0,5


We cannot drop single values from a `DataFrame`; we can only drop full rows or full columns.

In [17]:
df.dropna(axis='columns')

,2
0,2
1,5
2,6


Need better control of this - can do so via the `how` or `thresh` parameters

In [18]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [19]:
df.dropna(axis='columns', how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [22]:
# specify a minimum number of non-null values to keep a row/column
df.dropna(axis='rows', thresh=3)

,0,1,2,3
1,2.0,3.0,5,NaN


## Filling null values

Pandas provides the `fillna()` method, which returns a copy of the array with the null values replaced.

In [23]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [24]:
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

In [25]:
data.fillna(method='ffill')

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

This specifies a forward-fill to propagate the previous value forward

In [26]:
data.fillna(method='bfill')

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

In [27]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [28]:
# specify an axis to fill along
df.fillna(method='ffill', axis=1)

# axis=1 means horizontal, axis=0 means vertical

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0


In [31]:
df2 = df.fillna(method='ffill', axis=1)
df2.isnull()

,0,1,2,3
0,False,False,False,False
1,False,False,False,False
2,True,False,False,False


In [32]:
df3 = df.fillna(method='bfill', axis=1)
df3

,0,1,2,3
0,1.0,2.0,2.0,NaN
1,2.0,3.0,5.0,NaN
2,4.0,4.0,6.0,NaN


Notice that if a previous value is not available during a forward fill, the NA value remains.

# Summary

Missing data is an issue.

Pandas offers elegant ways of handling it:

- Find
- Drop
- Fill